In [2]:
def tokenize (chars):
    'Converte a string de caracteres em uma lista de tokens'
    return chars.replace('(', ' ( ').replace(')', ' ) ').split()

In [3]:
def parse(program: str):
    "Lê uma expressão de uma String"
    return read_from_tokens(tokenize(program))

In [4]:
def read_from_tokens (tokens: list): 
    "Lê uma expressão de uma sequência de tokens"
    if (len(tokens) == 0):
        raise SyntaxError ('unexpected EOF')
    token = tokens.pop(0)
    if token == '(':
        L = []
        while tokens[0] != ')':
            L.append(read_from_tokens(tokens))
        tokens.pop(0)
        return L
    elif token == ')':
        raise SyntaxError ('unexpected )')
    else:
        return atom(token)
        

In [5]:
def atom(token):
    "Trasnforma números em números e tokens em simbolos"
    try: 
        return float(token)
    except ValueError:
        return token

In [6]:
import math
import operator as op

In [7]:
def standard_env():
    env = {}
    env.update(vars(math))
    env.update({
        '+':op.add, 
        '-':op.sub, 
        '*':op.mul, 
        '/':op.truediv, 
        '>':op.gt,
        '<':op.lt,
        '>=':op.ge,
        '<=':op.le,
        '=':op.eq, 
        'abs':     abs,
        'append':  op.add,  
        'apply':   lambda proc, args: proc(*args),
        'begin':   lambda *x: x[-1],
        'car':     lambda x: x[0],
        'cdr':     lambda x: x[1:], 
        'cons':    lambda x,y: [x] + y,
        'eq?':     op.is_, 
        'expt':    pow,
        'equal?':  op.eq, 
        'length':  len, 
        'list':    lambda *x: list(x), 
        'list?':   lambda x: isinstance(x, list), 
        'map':     map,
        'max':     max,
        'min':     min,
        'not':     op.not_,
        'null?':   lambda x: x == [], 
        'number?': lambda x: isinstance(x, float),  
        'print':   print,
        'procedure?': callable,
        'round':   round,
        'symbol?': lambda x: isinstance(x, str),
    })
    return env
global_env = standard_env()
    
    

In [8]:
def eval(x, env=global_env):
    if isinstance(x, str):       
        return env[x]
    elif isinstance(x, float):     
        return x                
    elif x[0] == 'if':               
        (_, test, conseq, alt) = x
        exp = (conseq if eval(test, env) else alt)
        return eval(exp, env)
    elif x[0] == 'define':          
        (_, symbol, exp) = x
        env[symbol] = eval(exp, env)
    else:                            
        proc = eval(x[0], env)
        args = [eval(arg, env) for arg in x[1:]]
        return proc(*args)

In [9]:
eval(parse("(begin (define r 10) (* pi (* r r)))"))


314.1592653589793

In [11]:
def repl(prompt='lis.py> '):
    while True:
        val = eval(parse(input(prompt)))
        if val is not None: 
            print(schemestr(val))

In [12]:
def schemestr(exp):
    "Convert a Python object back into a Scheme-readable string."
    if isinstance(exp, list):
        return '(' + ' '.join(map(schemestr, exp)) + ')' 
    else:
        return str(exp)

In [ ]:
repl()